In [8]:
%pip install cmake 'gym[atari]' scipy gym

Note: you may need to restart the kernel to use updated packages.


In [9]:
import gym

env = gym.make("Taxi-v3", render_mode="ansi").env
env.reset( )
print(env.render())

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




In [10]:
env.reset()
env.render()

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

Action Space Discrete(6)
State Space Discrete(500)


In [41]:
env.s = 328 # set environment to illustration's state

epochs = 0
penalties, reward = 0, 0

frames = []

done = False

while not done :
    action = env.action_space.sample() # 택시가 각 스텝에서 무작위 행동 선택
    state, reward, done, info, _ = env.step(action)
    
    if reward == -10:
        penalties += 1
        
        frames.append({
            'frame' : env.render(),
            'state' : state,
            'action' : action,
            'reward' : reward
        })
        epochs += 1
        
print('Timesteps taken: {}'.format(epochs))
print('Penalties incurred : {}'.format(penalties))


Timesteps taken: 62
Penalties incurred : 62


In [43]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f'Timestep: {i + 1}')
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)

print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)

Timestep: 62
State: 418
Action: 4
Reward: -10


In [44]:
import numpy as np

q_tabel = np.zeros([env.observation_space.n, env.action_space.n])

state = env.reset()
type(state)
print(state)
print(state[1]['action_mask'])

(181, {'prob': 1.0, 'action_mask': array([1, 1, 0, 1, 0, 0], dtype=int8)})
[1 1 0 1 0 0]


In [45]:
# Q - Learning 코드 외우기 !!!
import random
from IPython.display import clear_output

# 하이퍼파라미터
alpha = 0.1  # 학습률
gamma = 0.6  # 할인 계수
epsilon = 0.1  # 탐색과 활용 사이의 균형을 맞추기 위한 입실론

# 플로팅을 위한 메트릭
all_epochs = []  # 각 에피소드별 시간(스텝)을 저장할 리스트
all_penalties = []  # 각 에피소드별 패널티를 저장할 리스트

# 100,000번의 에피소드를 반복하여 학습
for i in range(1, 100001):
    state = env.reset()  # 환경 초기화
    state = env.s  # 환경의 초기 상태 설정
    
    # 에피소드별 변수 초기화
    epochs, penalties, reward = 0, 0, 0
    done = False
    
    # 에피소드가 끝날 때까지 반복
    while not done:
        # 입실론-그리디 정책에 따라 행동 결정
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample()  # 탐색: 무작위 행동 선택
        else:
            action = np.argmax(q_tabel[state])  # 활용: 학습된 가치를 기반으로 행동 선택
        
        # 선택한 행동을 환경에 적용
        next_state, reward, done, info , _ = env.step(action)
        
        # Q-테이블을 업데이트
        old_value = q_tabel[state, action]
        next_max = np.max(q_tabel[next_state])
        
        # 새로운 Q-가치 계산
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_tabel[state, action] = new_value
        
        # 패널티 체크
        if reward == -10:
            penalties += 1
        
        # 상태와 스텝 수 업데이트
        state = next_state
        epochs += 1
        
    # 로그 출력
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode : {i}")

print("Training finished.\n")  # 학습 완료 메시지 출력


Episode : 100000
Training finished.



In [46]:
q_tabel[328]

array([ -2.45575536,  -2.46463913,  -2.45037881,  -2.46483584,
       -10.92842574, -11.34750765])

In [47]:
total_epochs, total_penalties = 0, 0
episodes = 10

for _ in range(episodes):
    state = env.reset()
    state = env.s
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        clear_output(wait=True)
        action = np.argmax(q_tabel[state])
        state, reward, done, info, _ = env.step(action)
        
        if reward == -10:
            penalties += 1
            
        epochs += 1
        print(env.render())
        sleep(.1)
        
    total_penalties += penalties
    total_epochs += epochs
    
print(f"Result after {episodes} episodes")
print(f"Average timesteps per episode : {total_epochs / episodes}")
print(f"Average penalties per episode : {total_penalties / episodes}")


+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Result after 10 episodes
Average timesteps per episode : 14.4
Average penalties per episode : 0.0
